<h1>Getting Started OpenAi with LangChain in Python Cookbook</h1>

The purpose of this notebook is to give a step by step explanation of how to get set-up with OpenAi and use available models through the LangChain Framework in Python. Furthermore, it will explain through examples how to implement models and available features.

<h2>Obtaining Keys and Endpoints</h2>

using the link - <a>https://azure.microsoft.com/en-us/pricing/purchase-options/azure-account</a> - one can sign up and create an OpenAI resource, giving access to all needed credentials.

<h2>Installing and Importing Dependencies</h2>

- install the most current version of LangChain and LangChain_OpenAi.
- import libraries/packages.

In [26]:
!pip install -U langchain langchain_openai

In [193]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import AzureChatOpenAI
import os
import getpass #use getpass for entering any environment variables or keys

<h4>Set all required Environment Variables</h4>

In [34]:
#API_KEY can be set as an environment variable securely like so
os.environ['OPENAI_API_KEY'] = getpass.getpass()
#os.environ['OPENAI_API_ENDPOINT'] = getpass.getpass()
#os.environ['OPENAI_API_VERSION'] = getpass.getpass()
#os.environ['COMPLETIONS_MODEL'] = getpass.getpass()
#...

<h4>Get all required Environment Variables</h4>

In [6]:
# Setting up the deployment name
DEPLOYMENT_NAME = os.environ['COMPLETIONS_MODEL']

# The API key for your Azure OpenAI resource.
API_KEY = os.environ['OPENAI_API_KEY']

# The base URL for your Azure OpenAI resource. e.g. "https://<your resource name>.openai.azure.com"
ENDPOINT = os.environ['OPENAI_API_ENDPOINT']

# The API version required
VERSION = os.environ['OPENAI_API_VERSION']

<h2>Creating a Chat Model</h2>

- environment variables can be manually passed as parameters
- the constuctor can search for environment variables of the corresponding names

Here <b>OPENAI_API_VERSION</b> and <b>OPENAI_API_ENDPOINT</b> are both passed, but <b>OPENAI_API_KEY</b> is being retrieved within the constructor

In [8]:
model = AzureChatOpenAI(
    openai_api_version=VERSION,  
    azure_deployment=DEPLOYMENT_NAME,
    azure_endpoint=ENDPOINT,
    temperature=0.5,
    max_tokens=None,
    timeout=60,
    max_retries=2,
    # organization="...",
    # model="gpt-35-turbo",
    # model_version="0125",
    # other params...
    )

<h4>Other Optional Parameters</h4>

- <b>temperature</b> determines how creative and unpredictable or how deterministic and predictive a model should try to be in it's responses. A temperature of 0 would be predictable and anything higher would become more random.

- <b>max_tokens</b> defines the maximum number of tokens (words or pieces of words) the model can generate in its response.

- <b>timeout</b> specifies the maximum amount of time (in seconds) to wait for a response from the API before timing out.

- <b>max_retries</b> sets the number of times the API request should be retried in case of failure before giving up.

- <b>organization</b> is used to specify the organization ID if required for API access.

- <b>model</b> specifies the model to be used.

- <b>model_version</b> indicates the specific version of the chosen model to use.

- Other parameters may be available in differen SDK's

<h2>Model Usage</h2>

<h4>Using Messages from the langchain_core.messages library allows the user to define messages for the model, as well as asign 'roles' to each of the message</h4>

In [11]:
messages = [
    SystemMessage(content="Translate the following from German into English"),
    HumanMessage(content="Sie haben gerade Ihr erstes Kunstliche Itelligenz Model erstellt!"),
]

In [13]:
response = model.invoke(messages)

In [15]:
response.content

'You have just created your first artificial intelligence model!'

<h4>Check the costs and token usage of a given model API call</h4>

In [17]:
from langchain.callbacks import get_openai_callback

In [19]:
messages = [
    SystemMessage(content="Translate the following from German into English"),
    HumanMessage(content="What's the first planet in the Solar System?"),
]

'get_openai_callback()' is a context manager of the OpenAICallbackHandler class found here - <a>https://github.com/langchain-ai/langchain/blob/a9e637b8f5a8873a8316ef6bf809591dc1f57c09/langchain/callbacks/openai_info.py</a>

In [39]:
with get_openai_callback() as cb:
    model.invoke(messages)
    print(f"Total tokens used: {cb.total_tokens}")
    print(f"Total prompt tokens: {cb.prompt_tokens}")
    print(f"Total prompt tokens: {cb.completion_tokens}")
    print(f"Total cost (in dollars): ${cb.total_cost}")
    print(f"Total successful requests): {cb.successful_requests}")

Total tokens used: 65
Total prompt tokens: 16
Total prompt tokens: 49
Total cost (in dollars): $0.000815
Total successful requests): 1


<h4>Async model usage</h4>

In [79]:
messages = [
    SystemMessage(content="Give a two sentance explanation of the following python code"),
    HumanMessage(content="await model.ainvoke(messages)"),
]

In [81]:
response = await model.ainvoke(messages)
response.content

'This Python code asynchronously invokes a method `ainvoke` on the `model` object, passing `messages` as an argument. It uses the `await` keyword, indicating that it is part of an asynchronous function and that it will pause execution until the `ainvoke` method completes.'

<h2>Tools and Structured output</h2>

Import the required packages. BaseModel is a parent class that all tools will inherit from, and Field is what all properties of the tool will be defined as. 

In [85]:
from langchain_core.pydantic_v1 import BaseModel, Field

Tools are essentially classes that can be passed to a chosen model to influence/structure how the response should be formatted or generated.

<b>For example:</b>

- A Weather tool with a specific API call could be passed so the model knows to use this specific API for data retrieval.
- A City tool with fields like population, size, main_language so the model can return any city related queries with an object containing the corresponding filled fields.
- An Image tool with a url field to be returned when asked to search for an image containing a dog with the field containing the url of the image.

In [101]:
from typing import Optional
#adds the ability to make a field Optional [NULL]

class Person(BaseModel):
    '''Information about a given person'''

    name: str = Field(..., description="The name of a person")
    alive: bool = Field(..., description="Thether the person is alive or not")
    profession: str = Field(..., description="What the person does for work or professionally")
    noteable_features: str = Field(..., description="Any noteworthy features/achievements about the person")
    hobbies: Optional[str] = Field(description="Anything hobbies the person may have")

In [122]:
structured_model = model.with_structured_output(Person)
response = structured_model.invoke("Tell me about Kate Sheppard")
response

Person(name='Kate Sheppard', alive=False, profession='Suffragist', noteable_features="Leader of the women's suffrage movement in New Zealand, instrumental in making New Zealand the first country to grant women the right to vote in 1893.", hobbies=None)

<h5>As the response of the invocation has been structured using the Person tool, the response can be accessed like a Person object.</h5>

In [111]:
print(response.name)
print(response.alive)
print(response.profession)
print(response.noteable_features)

Kate Sheppard
False
Suffragist
She was the most prominent member of New Zealand's women's suffrage movement and is widely credited with making New Zealand the first country to grant women the right to vote in 1893.


<h4>JSON</h4>

Models can also be explicitly told to respond in a JSON structured format. This could then be used for future API calls or simply easier access of information. However, <b>the word "json" must be included in the message string.</b>

In [152]:
json_model = model.bind(response_format={"type": "json_object"})

response = json_model.invoke(
    '''Return a JSON object of a random person with features like name,
    alive (if they're alive or not) and their profession.''' 
)

response.content

'{\n  "name": "Alexandra Johnson",\n  "alive": true,\n  "profession": "Software Engineer"\n}'

The response can then be formatted into JSON object and accessed using normal JSON notation

In [155]:
person = json.loads(response.content)
person

{'name': 'Alexandra Johnson', 'alive': True, 'profession': 'Software Engineer'}

<h2>File input</h2>

Models can be fed files of various forms as their inputs.

<h4>Images</h4>

Import the required libraries to allow for HTTP requests and file conversion.

Make sure the file format is correct otherwise a File not found error will be thrown.

In [181]:
import base64
import httpx

url = "https://upload.wikimedia.org/wikipedia/commons/b/bf/Aoraki_Mount_Cook.JPG"

# use an HTTP get request to retrieve the binary image data
binary_image_data = httpx.get(url).content

# encode the binary_image_data into base 64 so that it's in bytes
# this allows it to be transmitted in JSON or text format
byte_image_data = base64.b64encode(binary_image_data)

# decode the data into UTF-8 to allow for to make the data workable
# web applications and models require it in a workable format
# UTF-8 is a fairly standardised format
image_data = byte_image_data.decode("utf-8")

In [207]:
messages = [  
    SystemMessage(content=[{"type": "text", "text": "describe the image location"}]),
    HumanMessage(
    content=[
        {
            "type": "image_url",
            "image_url": {"url": f"data:image/jpeg;base64,{image_data}"},
        },
    ]
    )
]
response = model.invoke(messages)
response.content

'The image depicts a stunning mountain landscape with snow-covered peaks. The mountains are reflected beautifully in a calm, clear lake in the foreground, creating a mirror-like effect. The sky is mostly clear with a few scattered clouds, and the overall scene is serene and picturesque. The rocky shoreline of the lake is visible in the foreground, adding to the natural beauty of the location. The combination of the snow-capped mountains, clear lake, and blue sky creates a breathtaking and tranquil environment.'